In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [69]:
url_inicial = 'https://abm.ojs.inecol.mx/index.php/abm/issue/archive/6'
url_root = 'https://abm.ojs.inecol.mx/index.php/abm/issue/archive/6'
r=requests.get(url_inicial)

In [70]:
soup = BeautifulSoup(r.text, 'html.parser')

In [71]:
box = soup.find('div', class_='flex_container issues_list')
volumen=box.findAll('div',class_='obj_issue_summary')
vol = [x.find('a').get('href')for x in volumen]
vol

['https://abm.ojs.inecol.mx/index.php/abm/issue/view/100',
 'https://abm.ojs.inecol.mx/index.php/abm/issue/view/99',
 'https://abm.ojs.inecol.mx/index.php/abm/issue/view/98',
 'https://abm.ojs.inecol.mx/index.php/abm/issue/view/97']

In [72]:
vol2 =[]
for i in vol: 
    url_inicial1=i 
    r1 = requests.get(url_inicial1)
    soup1 = BeautifulSoup(r1.text, 'html.parser')
    box1 = soup1.find('div', class_='section_content') 
    volumen1=box1.findAll('article', class_='article_summary')
    vol1 = [x.find('a').get('href')for x in volumen1]
    vol2+=vol1

In [73]:
vol2

['https://abm.ojs.inecol.mx/index.php/abm/article/view/568',
 'https://abm.ojs.inecol.mx/index.php/abm/article/view/569',
 'https://abm.ojs.inecol.mx/index.php/abm/article/view/570',
 'https://abm.ojs.inecol.mx/index.php/abm/article/view/571',
 'https://abm.ojs.inecol.mx/index.php/abm/article/view/565',
 'https://abm.ojs.inecol.mx/index.php/abm/article/view/566',
 'https://abm.ojs.inecol.mx/index.php/abm/article/view/567',
 'https://abm.ojs.inecol.mx/index.php/abm/article/view/561',
 'https://abm.ojs.inecol.mx/index.php/abm/article/view/562',
 'https://abm.ojs.inecol.mx/index.php/abm/article/view/563',
 'https://abm.ojs.inecol.mx/index.php/abm/article/view/564',
 'https://abm.ojs.inecol.mx/index.php/abm/article/view/556']

In [74]:
def get_url_items(sopa,url):
    box1 = soup1.find('div', class_='section_content') 
    volumen1=box1.findAll('article', class_='article_summary')
    vol1 = [x.find('a').get('href')for x in volumen1]
    return vol2

In [75]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break

Estoy en la pagina https://abm.ojs.inecol.mx/index.php/abm/issue/archive/6


In [76]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

12

In [77]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [78]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Acta Botánica Mexicana'
    a='II. Biología y Química'
    tem='Botánica'
    s_item=BeautifulSoup(r.text,'html.parser')
    #titulo de revista
    try:
        titulo=tire
        content_book['Titulo Revista']=tire
    except AttributeError:
        content_book['Titulo Revista']=None
    #area
    try:
        area=a
        content_book['Area']=a
    except AttributeError:
        content_book['Area']=None
    #tematica
    try:
        tema=tem
        content_book['Tematica']=tem
    except AttributeError:
        content_book['Tematica']=None
    #titulo articulo
    try:
        titu=s_item.find('h1', class_='page_title article-full-title').get_text(strip=True)
        content_book['Titulo Articulo']=titu
    except AttributeError:
        content_book['Titulo Articulo']=None
    #resumen
    try:
        resu=s_item.find('div', class_='abstract').find('p').get_text(strip=True)
        content_book['Resumen']=resu
    except AttributeError:
        content_book['Resumen']=None
    #abstract
    try:
        abst=s_item.find('div', class_='item cover_image').get_text(strip=True)
        content_book['Abstract']=abst
    except AttributeError:
        content_book['Abstract']=None
    #Link incial
    try:
        linkart=s_item.find('div', class_='article_issue_credentials').find('a').get('href')
        content_book['Link Articulo']=urljoin(url_root, linkart)
    except AttributeError:
        content_book['Link Articulo']=None
    #link articulo
    try:
        link=s_item.find('a', class_='btn galley-link obj_galley_link pdf').get('href')
        content_book['Link PDF']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link PDF']=None
    return content_book

In [79]:
list_scraper=list_scraper[0:143]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2
estas escrapeando la pag 3
estas escrapeando la pag 4
estas escrapeando la pag 5
estas escrapeando la pag 6
estas escrapeando la pag 7
estas escrapeando la pag 8
estas escrapeando la pag 9
estas escrapeando la pag 10
estas escrapeando la pag 11


In [80]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo Revista,Área,Temática,Titulo Articulo,Resumen,Abstract,Link Articulo,Link PDF
0,Acta Botánica Mexicana,II. Biología y Química,Botánica,"El palo escrito, árbol de madera preciosa - un...","Se describeDalbergiapalo-escritosp. n., planta...",None,https://abm.ojs.inecol.mx/index.php/abm/issue/...,https://abm.ojs.inecol.mx/index.php/abm/articl...
1,Acta Botánica Mexicana,II. Biología y Química,Botánica,Primer registro de Ripartitella brasiliensis (...,"Se describe por primera vez, de la micoflora m...",None,https://abm.ojs.inecol.mx/index.php/abm/issue/...,https://abm.ojs.inecol.mx/index.php/abm/articl...
2,Acta Botánica Mexicana,II. Biología y Química,Botánica,Una nueva especie de Hyperbaena (Menispermacea...,Se describe una nueva especie de Menispermacea...,None,https://abm.ojs.inecol.mx/index.php/abm/issue/...,https://abm.ojs.inecol.mx/index.php/abm/articl...
3,Acta Botánica Mexicana,II. Biología y Química,Botánica,Morfología de los granos de polen de la famili...,Se estudia al microscopio de luz y al microsco...,None,https://abm.ojs.inecol.mx/index.php/abm/issue/...,https://abm.ojs.inecol.mx/index.php/abm/articl...
4,Acta Botánica Mexicana,II. Biología y Química,Botánica,Tres especies nuevas de Stachys (Labiatae) de ...,"Se describen como nuevasStachysherrerana, del ...",None,https://abm.ojs.inecol.mx/index.php/abm/issue/...,https://abm.ojs.inecol.mx/index.php/abm/articl...
5,Acta Botánica Mexicana,II. Biología y Química,Botánica,Análisis de la distribución geográfica del com...,Se ilustra y se discute la distribución geográ...,None,https://abm.ojs.inecol.mx/index.php/abm/issue/...,https://abm.ojs.inecol.mx/index.php/abm/articl...
6,Acta Botánica Mexicana,II. Biología y Química,Botánica,Dos nuevas especies de Pinguicula (Lentibulari...,Se describen dos especies nuevas dePinguiculad...,None,https://abm.ojs.inecol.mx/index.php/abm/issue/...,https://abm.ojs.inecol.mx/index.php/abm/articl...
7,Acta Botánica Mexicana,II. Biología y Química,Botánica,Primer registro de Trichoglossum velutipes (Ge...,Como parte de los estudios de hongos en el Est...,None,https://abm.ojs.inecol.mx/index.php/abm/issue/...,https://abm.ojs.inecol.mx/index.php/abm/articl...
8,Acta Botánica Mexicana,II. Biología y Química,Botánica,Descubrimiento de Pitcairnia undulata Scheidwe...,Se registra por primera vez la presencia dePit...,None,https://abm.ojs.inecol.mx/index.php/abm/issue/...,https://abm.ojs.inecol.mx/index.php/abm/articl...
9,Acta Botánica Mexicana,II. Biología y Química,Botánica,Tres especies nuevas de Karwinskia (Rhamnaceae...,Se describen tres especies nuevas deKarwinskia...,None,https://abm.ojs.inecol.mx/index.php/abm/issue/...,https://abm.ojs.inecol.mx/index.php/abm/articl...


In [81]:
df_catalogo.to_csv('Revista29.06.csv', index=False)